In [23]:
import numpy as np
import pandas as pd
import sklearn.linear_model
import sklearn.neighbors
import matplotlib.pyplot as plt

In [ ]:
def prepare_country_stats(oecd_bli, gdp_per_capita):
    oecd_bli = oecd_bli[oecd_bli["INEQUALITY"]=="TOT"]
    oecd_bli = oecd_bli.pivot(index="Country", columns="Indicator", values="Value")
    gdp_per_capita.rename(columns={"2015": "GDP per capita"}, inplace=True)
    gdp_per_capita.set_index("Country", inplace=True)
    full_country_stats = pd.merge(left=oecd_bli, right=gdp_per_capita,
                                  left_index=True, right_index=True)
    full_country_stats.sort_values(by="GDP per capita", inplace=True)
    remove_indices = [0, 1, 6, 8, 33, 34, 35]
    keep_indices = list(set(range(36)) - set(remove_indices))
    keep_indices = list(set(range(36)))
    training_set = full_country_stats[["GDP per capita", 'Life satisfaction']].iloc[keep_indices]
    test_set = full_country_stats[["GDP per capita", 'Life satisfaction']].iloc[remove_indices]
    return training_set, test_set

In [ ]:
# Load the data
datapath = '../handson-ml-master/datasets/lifesat/'

oecd_bli = pd.read_csv(datapath + "oecd_bli_2015.csv", thousands=',')
gdp_per_capita = pd.read_csv(datapath + "gdp_per_capita.csv",thousands=',',delimiter='\t',
                             encoding='latin1', na_values="n/a")

# Prepare the data
tr_set, te_set = prepare_country_stats(oecd_bli, gdp_per_capita)
X = np.c_[tr_set["GDP per capita"]]
y = np.c_[tr_set["Life satisfaction"]]

In [16]:
lin_reg_model = sklearn.linear_model.LinearRegression()
lin_reg_model.fit(X, y)

LinearRegression()

In [32]:
clf = sklearn.neighbors.KNeighborsRegressor(n_neighbors=2)
clf.fit(X, y)

KNeighborsRegressor(n_neighbors=2)

In [33]:
lin_yp = lin_reg_model.predict(np.c_[te_set["GDP per capita"]])
clf_yp = clf.predict(np.c_[te_set["GDP per capita"]])

In [34]:
te_set['lin_yp'] = lin_yp
te_set['clf_yp'] = clf_yp

In [35]:
te_set

,GDP per capita,Life satisfaction,lin_yp,clf_yp
Country,,,,
Brazil,8669.998,7.0,5.963977,6.85
Mexico,9009.280,6.7,5.971841,6.35
Chile,13340.905,6.7,6.072237,6.25
Czech Republic,17256.918,6.5,6.162999,6.05
Norway,74822.106,7.4,7.497207,7.45
Switzerland,80675.308,7.5,7.632869,7.45
Luxembourg,101994.093,6.9,8.126981,7.20
